# Preamble

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
sys.path.append('../SatLib')
# import satPlotTools as spt
import satbox as sb
# import satbox_MJD as sb_MJD
import orbitalMechanics as om
import utils as utils
# import utils_MJD as utils_MJD
import CZMLExtractor_MJD as CZMLExtractor_MJD


In [3]:
import numpy as np
from poliastro import constants
from poliastro.earth import Orbit
from poliastro.earth.sensors import min_and_max_ground_range, ground_range_diff_at_azimuth
from poliastro.bodies import Earth
from poliastro.maneuver import Maneuver
from poliastro.twobody.propagation import propagate
from poliastro.twobody.propagation import cowell
from poliastro.core.perturbations import J2_perturbation
from poliastro.core.propagation import func_twobody
from poliastro.util import norm
import astropy
import astropy.units as u
from astropy.time import Time, TimeDelta
from astropy.coordinates import Angle

In [4]:
import matplotlib.pyplot as plt
from poliastro.plotting.static import StaticOrbitPlotter
from poliastro.plotting import OrbitPlotter3D, OrbitPlotter2D
%matplotlib notebook

# Test constellation propagation

## Create constellation object

In [5]:
# %debug
i = 60 * u.deg
t = 4
p = 2
f = 1
alt = 600 * u.km
walker = sb.Constellation.from_walker(i, t, p, f, alt)

/Users/jacelu/opt/anaconda3/envs/SatelliteExample/lib/python3.9/site-packages/astropy/units/decorators.py:253: UserWarning: Wrapping true anomaly to -π <= nu < π
  return_ = wrapped_function(*func_args, **func_kwargs)


### Create constellation sim object

In [6]:
t2propagate = 5 * u.day
tStep = 60 * u.s
walkerSim = sb.SimConstellation(walker, t2propagate, tStep, verbose = True)

### Propagate

In [7]:
# %debug
walkerSim.propagate()

### Plot 3D Initial Satellites

In [8]:
walkerSim.constellation.planes[0].sats[0].rvECEF.obstime.isot

array(['2000-01-01T12:00:00.000', '2000-01-01T12:01:00.000',
       '2000-01-01T12:02:00.000', ..., '2000-01-06T11:57:00.000',
       '2000-01-06T11:58:00.000', '2000-01-06T11:59:00.000'], dtype='<U23')

In [9]:
op = walkerSim.initConstellation.plot_sats([0,1,2,3])
op.show()

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


## Extract data from simulated constellation

In [10]:
# outputData = walkerSim.get_relative_velocity_analysis()

## Getting InterSatellite Links

In [11]:
import Interval_Finder as IF 


In [12]:
relative_position_data_ISL = IF.get_relative_position_data_ISL(walkerSim,2000)

/Users/jacelu/opt/anaconda3/envs/SatelliteExample/lib/python3.9/site-packages/astropy/units/quantity.py:486: RuntimeWarning:

invalid value encountered in arccos



In [13]:
satellites = IF.find_feasible_links_ISL(4, relative_position_data_ISL)

In [14]:
L_avail_ISL = IF.get_availability_ISL(satellites, relative_position_data_ISL)

In [15]:
L_poly_ISl = IF.get_polyline_ISL(satellites, relative_position_data_ISL)
print(L_poly_ISl)

[[{'interval': '2000-01-01T12:00:00.000Z/2000-01-05T00:23:00.000Z', 'boolean': 'false'}, {'interval': '2000-01-05T00:23:00.000Z/2000-01-05T00:24:00.000Z', 'boolean': 'true'}, {'interval': '2000-01-05T00:24:00.000Z/2000-01-05T01:11:00.000Z', 'boolean': 'false'}, {'interval': '2000-01-05T01:11:00.000Z/2000-01-05T01:13:00.000Z', 'boolean': 'true'}, {'interval': '2000-01-05T01:13:00.000Z/2000-01-05T01:59:00.000Z', 'boolean': 'false'}, {'interval': '2000-01-05T01:59:00.000Z/2000-01-05T02:01:00.000Z', 'boolean': 'true'}, {'interval': '2000-01-05T02:01:00.000Z/2000-01-05T02:48:00.000Z', 'boolean': 'false'}, {'interval': '2000-01-05T02:48:00.000Z/2000-01-05T02:49:00.000Z', 'boolean': 'true'}, {'interval': '2000-01-05T02:49:00.000Z/2000-01-05T03:36:00.000Z', 'boolean': 'false'}, {'interval': '2000-01-05T03:36:00.000Z/2000-01-05T03:38:00.000Z', 'boolean': 'true'}, {'interval': '2000-01-05T03:38:00.000Z/2000-01-05T04:24:00.000Z', 'boolean': 'false'}, {'interval': '2000-01-05T04:24:00.000Z/2000-01

In [16]:
GS_pos = [[-71,42], [-86,21]] 

In [17]:
relative_position_data_GS = IF.get_relative_position_data_GS(walkerSim, GS_pos, 20)
print(relative_position_data_GS)

{'0-GS0': array([False, False, False, ..., False, False, False]), '0-GS1': array([False, False, False, ..., False, False, False]), '1-GS0': array([False, False, False, ..., False, False, False]), '1-GS1': array([False, False, False, ..., False, False, False]), '2-GS0': array([False, False, False, ..., False, False, False]), '2-GS1': array([False, False, False, ..., False, False, False]), '3-GS0': array([False, False, False, ..., False, False, False]), '3-GS1': array([False, False, False, ..., False, False, False]), 'times': array(['2000-01-01T12:00:00.000', '2000-01-01T12:01:00.000',
       '2000-01-01T12:02:00.000', ..., '2000-01-06T11:57:00.000',
       '2000-01-06T11:58:00.000', '2000-01-06T11:59:00.000'], dtype='<U23')}


In [18]:
objects = IF.find_feasible_links_GS(relative_position_data_GS)
print(objects)

['0-GS0', '0-GS1', '1-GS0', '1-GS1', '2-GS0', '2-GS1', '3-GS0', '3-GS1']


In [19]:
L_avail_GS = IF.get_availability_GS(objects, relative_position_data_GS)

In [20]:
L_poly_GS = IF.get_polyline_GS(objects, relative_position_data_GS)

In [22]:
walker.generate_czml_file(prop_duration=1, fname = 'visualizer', sample_points=100, satellites=satellites,objects=objects, L_avail_ISL=L_avail_ISL, L_poly_ISL=L_poly_ISl, L_avail_GS=L_avail_GS, L_poly_GS=L_poly_GS, GS_pos=GS_pos, GS=True, show_polyline_ISL=True, show_polyline_GS=True, create_file=True)